In [1]:
library(tidyverse)
library(lubridate)
library(corrplot)

set.seed(1992)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

corrplot 0.84 loaded


In [2]:
ipr_scan <- read.csv('../vitvi.vcostv3.clean.pep.txt', sep='\t')

## need to filter for genes tested in DGE analysis
ipr_sub <- ipr_scan %>% 
  select(genename, e.value, ipr_accession) %>%
  filter(e.value < 1e-10) %>% 
  filter(ipr_accession != '') %>%
  select(genename, ipr_accession) %>%
  unique()

#also grab pfam
pfam_sub <- ipr_scan %>% 
  select(genename, analysis, signature_accession, signature_description, e.value, ipr_accession) %>%
  filter(e.value < 1e-10) %>% 
  filter(analysis == 'Pfam') %>%
  select(genename, signature_accession, signature_description, ipr_accession) %>%
  unique()

In [3]:
full_enrichment_test <- function(set_of_genes, ipr_sub, print_res=FALSE){
    ipr_testset <- ipr_sub[ipr_sub$genename %in% set_of_genes,]
    terms <- unique(ipr_testset$ipr_accession)
    
    print(str_interp("analyzing ${length(terms)} IPR accessions..."))
    
    enriched_terms <- c() 
    for (term in terms){
        
        genes_to_term_in_set <- nrow(ipr_testset[ipr_testset$ipr_accession == term,])
        genes_to_term_in_genome <- nrow(ipr_sub[ipr_sub$ipr_accession == term,])
        genes_not_term_in_genome <- 24769 - genes_to_term_in_genome
        size_of_set <- length(set_of_genes)
         
        test <- phyper(genes_to_term_in_set, genes_to_term_in_genome,
                       genes_not_term_in_genome, size_of_set, lower.tail=FALSE)
        
        if (test < 0.05/length(terms) & 
            genes_to_term_in_genome >= 10 & 
            genes_to_term_in_set >= 3){
            if (print_res == TRUE) {
                print(c(term, test, genes_to_term_in_set, genes_to_term_in_genome))
            }
            enriched_terms <- c(enriched_terms, term, genes_to_term_in_set)
        }
    }
    
    return(enriched_terms)

}

full_pfam_enrichment_test <- function(set_of_genes, pfam_sub, print_res=FALSE){
    pfam_testset <- pfam_sub[pfam_sub$genename %in% set_of_genes,]
    terms <- unique(pfam_testset$signature_accession)
    
    print(str_interp("analyzing ${length(terms)} Pfam domains..."))
    
    enriched_terms <- c()
    for (term in terms){
        
        genes_to_term_in_set <- nrow(pfam_testset[pfam_testset$signature_accession == term,])
        genes_to_term_in_genome <- nrow(pfam_sub[pfam_sub$signature_accession == term,])
        genes_not_term_in_genome <- 24769 - genes_to_term_in_genome
        size_of_set <- length(set_of_genes)
        
        test <- phyper(genes_to_term_in_set, genes_to_term_in_genome,
                       genes_not_term_in_genome, size_of_set, lower.tail=FALSE)
        
        if (test < 0.05/length(terms) & 
            genes_to_term_in_genome >= 10 & 
            genes_to_term_in_set >= 3){
            if (print_res == TRUE){
                print(c(term, test, genes_to_term_in_set, genes_to_term_in_genome))
            }
            enriched_terms <- c(enriched_terms, term, genes_to_term_in_set)
        }
    }
    
    return(enriched_terms)

}

In [58]:
length(rownames(read.csv('all_rootstock_comparisons/Leaf_2018_Harvest_Ungrafted-1103P.csv')))

[1] 24770

In [6]:
## make whole analysis into a function

functional_enrichment <- function(cluster_id, ipr_scan, ipr_sub, pfam_sub){
    
    # get high loading genes
    set_genes <- read.csv('1719_geneExpression_SOM_clusters.csv')
    set_genes <- set_genes %>% 
      filter(cluster==cluster_id) %>%
      .[, 'gene']
    
    set_genes <- paste(set_genes, '.t01', sep='')
    
    all_genes <- read.csv('all_rootstock_comparisons/Leaf_2018_Harvest_Ungrafted-1103P.csv')[, 'X']
    all_genes <- paste(all_genes, '.t01', sep='')
    
    #filter for genes in DEG analysis
    ipr_sub_mtv <- ipr_sub %>%
      filter(genename %in% all_genes)

    pfam_sub_mtv <- pfam_sub %>%
      filter(genename %in% all_genes)

    #run ipr and pfam enrichments for a cluster
    eset_up <- full_enrichment_test(set_genes, ipr_sub_mtv)
    pset_up <- full_pfam_enrichment_test(set_genes, pfam_sub_mtv)

    
    #make output tables
    ipr_up <- ipr_scan %>%
      select(ipr_accession, ipr_description) %>%
      unique() %>%
      filter(ipr_accession %in% eset_up)
    
    pfam_up <- ipr_scan %>%
      filter(analysis == 'Pfam') %>%
      select(signature_accession, signature_description) %>%
      unique() %>%
      filter(signature_accession %in% pset_up)
    
    output <- list()
    output[['full_eset_up']][['enriched_terms']] <- eset_up
    output[['full_pset_up']][['enriched_terms']] <- pset_up
    
    output[['pretty_eset_up']][['enriched_terms']] <- ipr_up
    output[['pretty_pset_up']][['enriched_terms']] <- pfam_up
    
    return(output)
}

In [15]:
## heavy PC loaders

x <- functional_enrichment(7, ipr_scan=ipr_scan, ipr_sub=ipr_sub, pfam_sub=pfam_sub)
x[['pretty_eset_up']][['enriched_terms']]
x[['pretty_pset_up']][['enriched_terms']]

[1] "analyzing 411 IPR accessions..."
[1] "analyzing 246 Pfam domains..."


ipr_accession,ipr_description
IPR001650,"Helicase, C-terminal"
IPR014001,"Helicase superfamily 1/2, ATP-binding domain"
IPR012677,Nucleotide-binding alpha-beta plait domain superfamily
IPR035979,RNA-binding domain superfamily
IPR000504,RNA recognition motif domain
IPR001214,SET domain
IPR031330,"Glycoside hydrolase 35, catalytic domain"
IPR038718,"SNF2-like, N-terminal domain superfamily"
IPR000330,"SNF2-related, N-terminal domain"
IPR001487,Bromodomain


signature_accession,signature_description
PF00271,Helicase conserved C-terminal domain
PF00076,"RNA recognition motif. (a.k.a. RRM, RBD, or RNP domain)"
PF00856,SET domain
PF01301,Glycosyl hydrolases family 35
PF00176,SNF2 family N-terminal domain
PF00439,Bromodomain
PF04408,Helicase associated domain (HA2)
PF07717,Oligonucleotide/oligosaccharide-binding (OB)-fold


In [7]:
# I AM ONLY DOING 2018
# but you can do more years by adding the years to the proper list
# change time points as necessary/desired

tissues <- c('Leaf')
years <- c('2018')
phenos <- c('Anthesis', 'Veraison', 'Harvest')
rootstocks <- c('Ungrafted', '1103P', '3309C', 'SO4')
prefix <- '/data/projects/julia.pratt/CS1_genomeSelection/all_rootstock_comparisons/'

stats<- list()

In [37]:
# cycle through all the data
for (t in tissues){
    for (y in years){
        for (p in phenos){
            for (name in c('pretty_eset_up', 'pretty_eset_down', 'pretty_pset_up', 'pretty_pset_down')) {
                # rootstock combos, only going through ungrafted-X
                for (j in 2:length(rootstocks)){
                    rs <- str_interp("Ungrafted-${rootstocks[j]}")
                    fullpath <- paste(prefix, paste(t, y, p, rs, sep='_'), '.csv', sep='')
                    fname <- paste(t,y,p,rs,name,sep="_")
                    fnameformat <- paste(t,y,p,rs,name,sep=" ")
                    d <- functional_enrichment(fullpath, ipr_scan=ipr_scan, ipr_sub=ipr_sub, pfam_sub=pfam_sub)
#                     stats[[fname]] <- d[[name]]
                }                
            }
        exit()
        }
    }
}

[1] "analyzing 154 IPR accessions..."
[1] "analyzing 85 Pfam domains..."
[1] "analyzing 22 IPR accessions..."
[1] "analyzing 11 Pfam domains..."
  ipr_accession ipr_description
1     IPR001471  AP2/ERF domain
  signature_accession signature_description
1             PF00847            AP2 domain
[1] ipr_accession   ipr_description
<0 rows> (or 0-length row.names)
[1] signature_accession   signature_description
<0 rows> (or 0-length row.names)
[1] "analyzing 464 IPR accessions..."
[1] "analyzing 266 Pfam domains..."
[1] "analyzing 290 IPR accessions..."
[1] "analyzing 176 Pfam domains..."
  ipr_accession           ipr_description
1     IPR000070 Pectinesterase, catalytic
  signature_accession signature_description
1             PF01095        Pectinesterase
  ipr_accession ipr_description
1     IPR001214      SET domain
  signature_accession signature_description
1             PF00856            SET domain
[1] "analyzing 549 IPR accessions..."
[1] "analyzing 265 Pfam domains..."
[1] "an

ERROR: Error in exit(): could not find function "exit"


In [ ]:
names(stats)
stats[['Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up']]

In [64]:
0.05/102

[1] 0.0004901961

In [12]:
# UG_1103P <- functional_enrichment('/data/projects/julia.pratt/CS1_genomeSelection/all_rootstock_comparisons/Leaf_2018_Harvest_Ungrafted-1103P.csv',
#                                      ipr_scan=ipr_scan, ipr_sub=ipr_sub, pfam_sub=pfam_sub)

# UG_3309C <- functional_enrichment('/data/projects/julia.pratt/CS1_genomeSelection/all_rootstock_comparisons/Leaf_2018_Harvest_Ungrafted-3309C.csv',
#                                      ipr_scan=ipr_scan, ipr_sub=ipr_sub, pfam_sub=pfam_sub)

# UG_SO4 <- functional_enrichment('/data/projects/julia.pratt/CS1_genomeSelection/all_rootstock_comparisons/Leaf_2018_Harvest_Ungrafted-SO4.csv',
#                                      ipr_scan=ipr_scan, ipr_sub=ipr_sub, pfam_sub=pfam_sub)

[1] "analyzing 2634 IPR accessions..."
[1] "analyzing 1570 Pfam domains..."
[1] "analyzing 1884 IPR accessions..."
[1] "analyzing 1139 Pfam domains..."
[1] signature_accession   signature_description
<0 rows> (or 0-length row.names)
[1] "analyzing 1991 IPR accessions..."
[1] "analyzing 1180 Pfam domains..."
[1] "analyzing 1205 IPR accessions..."
[1] "analyzing 723 Pfam domains..."
[1] signature_accession   signature_description
<0 rows> (or 0-length row.names)
[1] "analyzing 2595 IPR accessions..."
[1] "analyzing 1567 Pfam domains..."
[1] "analyzing 1603 IPR accessions..."
[1] "analyzing 959 Pfam domains..."
[1] signature_accession   signature_description
<0 rows> (or 0-length row.names)


In [8]:
plotvenn <- function(rs1103p, rs3309c, rsso4, fname, fnameformat){
    # change the name of the pdf saved based on what kind of venn is being generated
    fileprefix= "/data/projects/julia.pratt/CS1_genomeSelection/figs/go/venn/venn_pset_"
    f <- paste(fileprefix, fname, '.png', sep="")
    
    # the function takes in three lists
    # automatically counts how many are shared/private within those lists
    # most of this is just styling
    venn.diagram(
        x = list(rs1103p, rs3309c, rsso4),
        
        cex = 0.5,
        fontfamily = "sans",
        
        category.names = c("1103P", "3309C", "SO4"),
        cat.cex = 0.5,
        cat.default.pos = "outer",
        cat.dist = c(0.075, 0.075, 0.075),
        cat.fontfamily = "sans",
        
        main = fnameformat,
        main.cex = 0.6,
        main.fontfamily = "sans",
        
        col=c("#1b9e77", "#7570b3", "#e6ab02"),
        fill = c(alpha("#1b9e77",0.3), alpha("#7570b3",0.3), alpha("#e6ab02",0.3)),
        
        filename = f,
        output=TRUE, 
        resolution = 300,
        imagetype="png",
        margin=0.075,
        rotation = 1,
        height=700,
        width=700
    )
}

In [ ]:
plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]][["ipr_accession"]], 
         "leaf_anth_2018_eset_up", "Leaf Anthesis 2018 ESET Up")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]][["ipr_accession"]], 
         "leaf_ver_2018_eset_up", "Leaf Veraison 2018 ESET Up")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]][["ipr_accession"]], 
         "leaf_harv_2018_eset_up", "Leaf Harvest 2018 ESET Up")

plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]][["ipr_accession"]], 
         "leaf_anth_2018_eset_down", "Leaf Anthesis 2018 ESET Down")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]][["ipr_accession"]], 
         "leaf_ver_2018_eset_down", "Leaf Veraison 2018 ESET Down")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]][["ipr_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]][["ipr_accession"]], 
         "leaf_harv_2018_eset_down", "Leaf Harvest 2018 ESET Down")

In [ ]:
plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], 
         "leaf_anth_2018_pset_up", "Leaf Anthesis 2018 PSET Up")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], 
         "leaf_ver_2018_pset_up", "Leaf Veraison 2018 PSET Up")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], 
         "leaf_harv_2018_pset_up", "Leaf Harvest 2018 PSET Up")

plotvenn(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], 
         "leaf_anth_2018_pset_down", "Leaf Anthesis 2018 PSET Down")
plotvenn(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], 
         "leaf_ver_2018_pset_down", "Leaf Veraison 2018 PSET Down")
plotvenn(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]],
         stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], 
         "leaf_harv_2018_pset_down", "Leaf Harvest 2018 PSET Down")

In [ ]:
pset.up.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]])
if(!is.null(pset.up.1103p)){
    d.pset.up.1103p = data.frame(gene=pset.up.1103p)
    d.pset.up.1103p = d.pset.up.1103p %>% arrange(gene)
    d.pset.up.1103p['color'] = viridis(length(pset.up.1103p))
    
    pal.ant <- d.pset.up.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P UP PSET')

    g.ver.u.1103p.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P UP PSET')

    g.har.u.1103p.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P UP PSET')
} else {
    print("no 1103p up")
}
pset.down.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]])
if(!is.null(pset.down.1103p)){
    d.pset.down.1103p = data.frame(gene=pset.down.1103p)
    d.pset.down.1103p = d.pset.down.1103p %>% arrange(gene)
    d.pset.down.1103p['color'] = viridis(length(pset.down.1103p))
    
    pal.ant <- d.pset.down.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P DOWN PSET')

    g.ver.u.1103p.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P DOWN PSET')

    g.har.u.1103p.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P DOWN PSET')
} else {
    print("no 1103p down")
}

pset.up.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]])
if(!is.null(pset.up.3309c)){
    d.pset.up.3309c = data.frame(gene=pset.up.3309c)
    d.pset.up.3309c = d.pset.up.3309c %>% arrange(gene)
    d.pset.up.3309c['color'] = viridis(length(pset.up.3309c))
    
    pal.ant <- d.pset.up.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C UP PSET')

    g.ver.u.3309c.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C UP PSET')

    g.har.u.3309c.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C UP PSET')
} else {
    print("no 3309c up")
}
pset.down.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]])
if(!is.null(pset.down.3309c)){
    d.pset.down.3309c = data.frame(gene=pset.down.3309c)
    d.pset.down.3309c = d.pset.down.3309c %>% arrange(gene)
    d.pset.down.3309c['color'] = viridis(length(pset.down.3309c))
    
    pal.ant <- d.pset.down.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C DOWN PSET')

    g.ver.u.3309c.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C DOWN PSET')

    g.har.u.3309c.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C DOWN PSET')
} else {
    print("no 3309c down")
}

pset.up.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]])
if(!is.null(pset.up.so4)){
    d.pset.up.so4 = data.frame(gene=pset.up.so4)
    d.pset.up.so4 = d.pset.up.so4 %>% arrange(gene)
    d.pset.up.so4['color'] = viridis(length(pset.up.so4))
    
    pal.ant <- d.pset.up.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 UP PSET')

    g.ver.u.so4.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 UP PSET')

    g.har.u.so4.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 UP PSET')
} else {
    print("no so4 up")
}
pset.down.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]])
if(!is.null(pset.down.so4)){
    d.pset.down.so4 = data.frame(gene=pset.down.so4)
    d.pset.down.so4 = d.pset.down.so4 %>% arrange(gene)
    d.pset.down.so4['color'] = viridis(length(pset.down.so4))
    
    pal.ant <- d.pset.down.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.pset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.pset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 DOWN PSET')

    g.ver.u.so4.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 DOWN PSET')

    g.har.u.so4.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_pset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 DOWN PSET')
} else {
    print("no so4 down")
}

In [ ]:
eset.up.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]])
if(!is.null(eset.up.1103p)){
    d.eset.up.1103p = data.frame(gene=eset.up.1103p)
    d.eset.up.1103p = d.eset.up.1103p %>% arrange(gene)
    d.eset.up.1103p['color'] = viridis(length(eset.up.1103p))
    
    pal.ant <- d.eset.up.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.up.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P UP ESET')

    g.ver.u.1103p.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P UP ESET')

    g.har.u.1103p.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P UP ESET')
} else {
    print("no 1103p up")
}
eset.down.1103p = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]])
if(!is.null(eset.down.1103p)){
    d.eset.down.1103p = data.frame(gene=eset.down.1103p)
    d.eset.down.1103p = d.eset.down.1103p %>% arrange(gene)
    d.eset.down.1103p['color'] = viridis(length(eset.down.1103p))
    
    pal.ant <- d.eset.down.1103p %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.down.1103p %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.1103p.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-1103P_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 1103P DOWN ESET')

    g.ver.u.1103p.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-1103P_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 1103P DOWN ESET')

    g.har.u.1103p.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-1103P_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 1103P DOWN ESET')
} else {
    print("no 1103p down")
}

eset.up.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]])
if(!is.null(eset.up.3309c)){
    d.eset.up.3309c = data.frame(gene=eset.up.3309c)
    d.eset.up.3309c = d.eset.up.3309c %>% arrange(gene)
    d.eset.up.3309c['color'] = viridis(length(eset.up.3309c))
    
    pal.ant <- d.eset.up.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.up.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C UP ESET')

    g.ver.u.3309c.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C UP ESET')

    g.har.u.3309c.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C UP ESET')
} else {
    print("no 3309c up")
}
eset.down.3309c = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]])
if(!is.null(eset.down.3309c)){
    d.eset.down.3309c = data.frame(gene=eset.down.3309c)
    d.eset.down.3309c = d.eset.down.3309c %>% arrange(gene)
    d.eset.down.3309c['color'] = viridis(length(eset.down.3309c))
    
    pal.ant <- d.eset.down.3309c %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.down.3309c %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.3309c.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-3309C_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS 3309C DOWN ESET')

    g.ver.u.3309c.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-3309C_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON 3309C DOWN ESET')

    g.har.u.3309c.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-3309C_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST 3309C DOWN ESET')
} else {
    print("no 3309c down")
}

eset.up.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]])
if(!is.null(eset.up.so4)){
    d.eset.up.so4 = data.frame(gene=eset.up.so4)
    d.eset.up.so4 = d.eset.up.so4 %>% arrange(gene)
    d.eset.up.so4['color'] = viridis(length(eset.up.so4))
    
    pal.ant <- d.eset.up.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.up.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.up <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 UP ESET')

    g.ver.u.so4.up <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 UP ESET')

    g.har.u.so4.up <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_up"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 UP ESET')
} else {
    print("no so4 up")
}
eset.down.so4 = union(union(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]], stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]), stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]])
if(!is.null(eset.down.so4)){
    d.eset.down.so4 = data.frame(gene=eset.down.so4)
    d.eset.down.so4 = d.eset.down.so4 %>% arrange(gene)
    d.eset.down.so4['color'] = viridis(length(eset.down.so4))
    
    pal.ant <- d.eset.down.so4 %>% 
    filter(gene %in% stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]) %>%
    select(color)

    pal.ver <- d.eset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    pal.har <- d.eset.down.so4 %>% 
        filter(gene %in% stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]][["signature_accession"]]) %>%
        select(color)

    g.ant.u.so4.down <- ggplot(stats[["Leaf_2018_Anthesis_Ungrafted-SO4_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ant[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('ANTHESIS SO4 DOWN ESET')

    g.ver.u.so4.down <- ggplot(stats[["Leaf_2018_Veraison_Ungrafted-SO4_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.ver[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('VERAISON SO4 DOWN ESET')

    g.har.u.so4.down <- ggplot(stats[["Leaf_2018_Harvest_Ungrafted-SO4_pretty_eset_down"]], aes(x = "" , y = intersection_size, fill = fct_inorder(signature_accession))) +
            geom_col(width = 1, color = 1) +
            scale_fill_manual(values=pal.har[['color']]) +
            theme_void() +
            theme(legend.position="none",
                  legend.margin = margin(25, 10, 10, 10)) +
            ggtitle('HARVEST SO4 DOWN ESET')
} else {
    print("no so4 down")
}

In [ ]:
# # ran this four times and changed pretty_eset_up, pretty_eset_down, pretty_pset_up, pretty_pset_down
# for (name in names(stats)) {
#     if(grepl("pretty_eset_up", name, fixed=TRUE)){
#         if(grepl("Leaf_2018_Anthesis_Ungrafted", name, fixed=TRUE)){
#             print(name)
#         }
#         if(grepl("Leaf_2018_Veraison_Ungrafted", name, fixed=TRUE)){
#             print(name)
#         }
#         if(grepl("Leaf_2018_Harvest_Ungrafted", name, fixed=TRUE)){
#             print(name)
#         }
#     }
# }